In [1]:
import numpy as np
import pandas as pd
import pickle

# 1. Import et traitement de la population par pays

In [2]:
# Read the data from CSV
df1 = pd.read_csv("data/population-2003-2013.csv")

# Select the useful columns + Pivot with years (2003 & 2013) + rename columns + rename the column axe
df1 = (pd
              .pivot_table(df1, values='Valeur', index=['Code Pays', 'Pays'],  columns='Année', aggfunc=sum)
              
              .reset_index()
  
              .rename(columns = {
                'Pays': 'country',
                'Code Pays': 'country_code',
                2003: 'pop_before',
                2013: 'pop_after',
              })
              
              .rename_axis(None, axis=1)
        )

# Compute the progression
df1['pop_evol'] = df1.pop_after - df1.pop_before
df1['pop_evol_ratio'] = df1.pop_evol / df1.pop_before

# Keep the useful columns
df_population = df1[['country', 'pop_evol_ratio']].copy()
display(df_population.head())

,country,pop_evol_ratio
0,Arménie,-0.019433
1,Afghanistan,0.321682
2,Albanie,-0.020377
3,Algérie,0.188013
4,Angola,0.392387


## Notes

J'ai d'abord choisi 2013 et 2012, puis j'ai finalement opté pour 2013 et 2003 pour que les écarts d'évolution entre les pays soient plus marqués. Ceci s'est confirmé en regardant la variance :

- Variance entre 2013 et 2012 : 0.00015339462894734522
- Variance entre 2013 et 2003 : 0.03175882395580265

---

# 2. Import et traitement de la disponibilité alimentaire totale

In [3]:
# Read the data from CSV
df2 = pd.read_csv("data/disponibilite-alimentaire-2013.csv")

# Select the useful columns + pivot table with 'elements' aggregated by sum, rename columns and axis
df2 = (pd
          .pivot_table(df2, values='Valeur', index=['Code Pays', 'Pays'], columns=['Élément'], aggfunc=np.sum)
       
           .reset_index()
       
           .rename(columns={
               'Code Pays':'country_code',
               'Pays': 'country',
               'Disponibilité alimentaire (Kcal/personne/jour)': 'kcal_total_capita_day',
               'Disponibilité de protéines en quantité (g/personne/jour)': 'proteins_total_capita_day'
           })
       
           .rename_axis(None, axis=1)
      )

# Keep the useful columns
df_food_availability_total = df2[['country','kcal_total_capita_day', 'proteins_total_capita_day']].copy()
df_food_availability_total.head(10)

,country,kcal_total_capita_day,proteins_total_capita_day
0,Arménie,2924.0,90.06
1,Afghanistan,2087.0,58.26
2,Albanie,3188.0,111.37
3,Algérie,3293.0,91.92
4,Angola,2474.0,57.27
5,Antigua-et-Barbuda,2416.0,83.49
6,Argentine,3226.0,102.60
7,Australie,3278.0,106.26
8,Autriche,3770.0,106.20
9,Bahamas,2670.0,86.16


# 3. Import et traitement de la disponibilité alimentaire animale

In [4]:
# Read the data from CSV
df3 = pd.read_csv("data/disponibilite-alimentaire-animale-2013.csv")

# Select the useful columns + pivot table with 'elements' aggregated by sum, rename columns and axis
df3 = (pd
          .pivot_table(df3, values='Valeur', index=['Code Pays', 'Pays'], columns=['Élément'], aggfunc=np.sum)
       
           .reset_index()
       
           .rename(columns={
               'Code Pays':'country_code',
               'Pays': 'country',
               'Disponibilité alimentaire (Kcal/personne/jour)': 'kcal_animal_capita_day',
               'Disponibilité de protéines en quantité (g/personne/jour)': 'proteins_animal_capita_day'
           })
       
           .rename_axis(None, axis=1)
      )

# Keep the useful columns
df_food_availability_animal = df3[['country', 'proteins_animal_capita_day']].copy()
df_food_availability_animal.head(10)

,country,proteins_animal_capita_day
0,Arménie,43.25
1,Afghanistan,12.22
2,Albanie,59.42
3,Algérie,24.99
4,Angola,18.40
5,Antigua-et-Barbuda,56.83
6,Argentine,66.94
7,Australie,71.68
8,Autriche,62.86
9,Bahamas,56.43


# 4. Construction et sauvegarde du dataframe principal

In [5]:
main_df = (
            # We use df_population as main dataframe
            df_population
                
                # We copy it not to modify the original one
                .copy()
    
                # We merge df_food_availability_total
                .merge(df_food_availability_total, how='left', on='country')
    
                # We merge df_food_availability_animal
                .merge(df_food_availability_animal, how='left', on='country')
    
                # We create a new column for the [animal proteins] / [total proteins] ratio
                .assign(
                    proteins_animal_ratio = lambda x: x.proteins_animal_capita_day / x.proteins_total_capita_day
                )
    
                # We drop columns that we don't need
                .drop(['proteins_animal_capita_day'], axis=1)
)

display(main_df.head(20))

# We save the main_df in a file
with open('data/part1.pkl', 'wb') as f:
    my_pickler = pickle.Pickler(f)
    my_pickler.dump(main_df)

,country,pop_evol_ratio,kcal_total_capita_day,proteins_total_capita_day,proteins_animal_ratio
0,Arménie,-0.019433,2924.0,90.06,0.480235
1,Afghanistan,0.321682,2087.0,58.26,0.209749
2,Albanie,-0.020377,3188.0,111.37,0.533537
3,Algérie,0.188013,3293.0,91.92,0.271867
4,Angola,0.392387,2474.0,57.27,0.321285
5,Antigua-et-Barbuda,0.111111,2416.0,83.49,0.680680
6,Argentine,0.091546,3226.0,102.60,0.652437
7,Australie,0.169899,3278.0,106.26,0.674572
8,Autriche,0.043355,3770.0,106.20,0.591902
9,Bahamas,0.193038,2670.0,86.16,0.654944
